#*Dự đoán điểm thi cuối kỳ của sinh viên môn nhập môn lập trình(IT001) thông qua quá trình nộp bài Wecode*

In [ ]:
import pandas as pd # Thư viện đọc file csv

data=pd.read_csv("wecode.csv")# Sau khi tải file wecode.csv lên colab ta đọc file
data.head() # Một vài dữ liệu đầu của file

,id,assignment_id,problem_id,user_id,is_final,status,pre_score,coefficient,file_name,language_id,created_at,updated_at
0,4,0,3,4,1,SCORE,10000,100,solution-0,2,10/30/2020 10:58,10/30/2020 10:58
1,5,0,4,4,1,SCORE,10000,100,solution-1,2,10/30/2020 11:07,10/30/2020 11:07
2,6,0,5,4,0,Compilation Error,0,100,solution-2,2,10/30/2020 11:08,10/30/2020 11:23
3,7,0,5,4,0,Compilation Error,0,100,solution-3,2,10/30/2020 11:11,10/30/2020 11:11
4,8,0,5,4,0,Compilation Error,0,100,solution-4,2,10/30/2020 11:15,10/30/2020 11:15


In [ ]:
# Kết hợp với điểm số từng sinh viên cùng với tất cả số lần nộp bài
df=data.groupby('user_id').agg(Diem =('pre_score', 'sum'), So_lan_nop_bai=('problem_id', 'count'))
df


,Diem,So_lan_nop_bai
user_id,,
1,10000,2
4,466000,54
5,30000,6
6,965637,130
7,18843,5
...,...,...
1860,405440,76
1862,40000,12
1864,563292,187


In [ ]:
table_score=pd.read_csv('score_training.csv')# Đọc file điểm CK
table_score

,id,QT,GK,TH,CK,TBM,TBTL
0,22,10,9,8.5,8.5,8.8,8.52
1,24,10,8,10,9.5,9.4,8.41
2,27,10,10,10,10,10,8.67
3,28,10,9.5,7.5,9.5,9.2,7.39
4,29,10,9.5,5,8.5,8.2,8.05
...,...,...,...,...,...,...,...
970,1857,5,4.5,10,4,6,6.03
971,1859,8,5,10,1.5,5.4,5.06
972,1862,5,6.5,1,7.5,5.1,4.71
973,1865,6.5,,3,7.5,,6.89


In [ ]:
# Xóa các dữ liệu thiếu
df1 = table_score.apply (pd.to_numeric, errors='coerce')
df1 = df1.dropna(how = 'any',axis= 0)
df1
df1=df1.rename(columns={'id' : 'user_id'})# Đối tên id thành user_id để kết với bảng wecode

In [ ]:
df1

,user_id,QT,GK,TH,CK,TBM,TBTL
0,22,10.0,9.0,8.5,8.5,8.8,8.52
1,24,10.0,8.0,10.0,9.5,9.4,8.41
2,27,10.0,10.0,10.0,10.0,10.0,8.67
3,28,10.0,9.5,7.5,9.5,9.2,7.39
4,29,10.0,9.5,5.0,8.5,8.2,8.05
...,...,...,...,...,...,...,...
969,1856,5.0,5.5,7.0,7.5,6.7,7.34
970,1857,5.0,4.5,10.0,4.0,6.0,6.03
971,1859,8.0,5.0,10.0,1.5,5.4,5.06
972,1862,5.0,6.5,1.0,7.5,5.1,4.71


In [ ]:
df2=pd.merge(df,df1,on='user_id') # Kết hai file với nhau thông qua user_id
df2

,user_id,Diem,So_lan_nop_bai,QT,GK,TH,CK,TBM,TBTL
0,24,1031674,164,10.0,8.0,10.0,9.5,9.4,8.41
1,27,52000,13,10.0,10.0,10.0,10.0,10.0,8.67
2,29,1225828,199,10.0,9.5,5.0,8.5,8.2,8.05
3,35,1809718,297,7.0,9.5,6.0,4.5,6.2,5.75
4,37,1550820,312,5.5,5.5,7.5,7.5,6.9,8.27
...,...,...,...,...,...,...,...,...,...
677,1771,834936,200,0.0,5.0,4.0,5.5,4.4,6.40
678,1856,329074,64,5.0,5.5,7.0,7.5,6.7,7.34
679,1857,389000,55,5.0,4.5,10.0,4.0,6.0,6.03
680,1859,39000,31,8.0,5.0,10.0,1.5,5.4,5.06


In [ ]:
df2=df2[['Diem','So_lan_nop_bai','CK']] # Bỏ những thuộc tính không liên quan giữ lại điểm CK để train
df2

,Diem,So_lan_nop_bai,CK
0,1031674,164,9.5
1,52000,13,10.0
2,1225828,199,8.5
3,1809718,297,4.5
4,1550820,312,7.5
...,...,...,...
677,834936,200,5.5
678,329074,64,7.5
679,389000,55,4.0
680,39000,31,1.5


In [ ]:
# Tiền xử lý dữ liệu
X=df2.iloc[:, 0:2] # đầu vào là mảng các giá trị của thuộc tính ảnh hưởng
y=df2.iloc[:, 2] # đầu ra là vector các điểm CK

In [ ]:
print('score_wecode, no. of submissions, Score_final') 
for i in range(10):
     print('{:8.0f}{:10.0f}{:20.0f}'.format(X.iloc[i,0], X.iloc[i,1], y[i]))

score_wecode, no. of submissions, Score_final
 1031674       164                  10
   52000        13                  10
 1225828       199                   8
 1809718       297                   4
 1550820       312                   8
 3356653       664                   8
 2411386       324                   8
 2378013       448                   6
 1284084       313                   6
 1636504       349                   4


In [ ]:
# Chia tập dữ liệu
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=1/3,random_state=0)

In [ ]:
# fitting bằng model regression
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train,y_train) #actually produces the linear eqn for the data

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
# Dự đoán các kết quả của tập test
y_pred = regressor.predict(X_test) 
y_pred
 


array([7.06494548, 6.00022381, 6.24024705, 7.87221952, 6.45482991,
       6.16879702, 6.44579135, 6.14895144, 5.43160167, 5.19056987,
       5.39558179, 6.64458768, 6.73626445, 6.0033281 , 6.14256057,
       6.49924518, 7.5716359 , 6.174468  , 7.94691462, 7.63264639,
       6.6527701 , 6.05041529, 5.59666228, 5.64603441, 5.74498677,
       6.67181772, 6.2816973 , 5.8112162 , 5.97736261, 6.68192244,
       5.79820103, 5.76556398, 6.72734171, 6.1180266 , 6.24197292,
       5.92024486, 5.78411537, 7.13066317, 8.50309701, 6.23233539,
       6.24482154, 5.31518511, 6.37417771, 5.98678559, 8.18079571,
       5.61318552, 5.61757404, 6.58532867, 8.26787865, 7.03215993,
       5.38880634, 6.2810418 , 6.08050602, 6.1459527 , 5.58403211,
       8.50824146, 5.67088011, 5.22165504, 6.7778866 , 5.32226203,
       5.43498253, 5.61034992, 5.20304494, 6.36529549, 5.86050242,
       6.32645167, 5.43571588, 6.65249625, 7.2162284 , 6.81201717,
       5.72263991, 7.22075749, 6.15835574, 5.62476213, 6.86539

In [ ]:
y_test

113    8.0
378    5.5
303    8.0
504    8.5
301    5.5
      ... 
132    6.5
173    8.0
178    9.0
630    8.0
344    5.0
Name: CK, Length: 228, dtype: float64

*Tính Mean Squared Error*

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)

3.806466227323717